In [1]:
import json
from utils.schema import *
from utils.reverse_logic import *
from utils.reverse_sql import *
from utils.reverse_middle import *
import os, sqlite3
import random
import copy

In [2]:
with open('raw_data/text_to_sql_data.json','r') as f:
    data = json.load(f)

with open('raw_data/tables.json','r') as f:
    db = json.load(f)
db_dict = {}
schemas, db_names, tables = get_schemas_from_json('raw_data/tables.json')
for item in db:
    db_dict[item['db_id']] = item

db_reverse = {}
db_c_ori2pre = {}
db_t_ori2pre = {}
for k,v in db_dict.items():
    db_reverse[k] = {}
    column = []
    table = []
    db_c_ori2pre[k] = {}
    db_t_ori2pre[k] = {}
    for o,c in zip(v['column_names'],v['column_names_original']):
        column.append([o[0],o[1],c[1]])
        db_c_ori2pre[k][c[1]] = o[1]
    db_reverse[k]['column'] = column
    for o,c in zip(v['table_names'],v['table_names_original']):
        table.append([o,c])
        db_t_ori2pre[k][c] = o
    db_reverse[k]['table'] = table
db_aug = {}
for k,v in db_dict.items():
    col_temp = {}
    for index, item in enumerate(v['column_names']):
        if item[0] not in col_temp.keys():
            if item[1] in v['primary_keys']:
                col_temp[item[0]] = [(index,item[1],v['column_types'][index],'primary')]
            else:
                col_temp[item[0]] = [(index,item[1],v['column_types'][index],'None')]
        else:
            if item[1] in v['primary_keys']:
                col_temp[item[0]].append((index,item[1],v['column_types'][index],'primary'))
            else:
                col_temp[item[0]].append((index,item[1],v['column_types'][index],'None'))
    db_aug[k] = col_temp

In [3]:
final = []
for index,item in enumerate(data):
    temp = {}
    db_id = item['db_id']
    if db_id in ['formula_1','store_1','scholar']:
        continue
    sql = item['query']
    schema = schemas[db_id]
    table = tables[db_id]
    schema = Schema(schema, table)
    try:
        sql_label = get_sql(schema, sql)
        temp['struct'] = sql_label
        flag = True
        for titem in temp['struct']['where']:
            if titem == 'and' or titem == 'or':
                continue
            elif isinstance(titem[3],tuple):
                flag = False

        temp['query'] = item['query']
        temp['question'] = item['question']
        for k, v in db_dict[db_id].items():
            temp[k] = v
        if flag:
            final.append(temp)
    except:
        print(index)
data_final = []
sql_all = []
for index,item in enumerate(final):
    try:
        if len(item['struct']['from']["table_units"]) > 1 and len(item['struct']['from']["conds"]) == 0:
            continue
        a = reverse(item['struct'],item['column_names'])
        logic = translate_logic(a,0,item['db_id'],db_reverse)
        sql = translate_sql(a,0,item['db_id'],db_reverse)
        temp = {}
        temp['db_id'] = item['db_id']
        temp['query_ori'] = item['query']
        temp['struct'] = item['struct'] 
        temp['logic'] = logic
        temp['query'] = sql
        temp['middle'] = a
        temp['question'] = item['question']
        if temp['query'] not in sql_all:
            data_final.append(temp)
            sql_all.append(temp['query'])
    except:
        print(index)
        pass


8314
10191
27207
34627
2719
3395
8120
8739
8740
9142
9143
9897
9898
10113
10114
10115
11053
11054
11055
11417
11418
12356
12357
14995
14996
18075
18076
18077
18108
18109
18110
18111
18164
18165
19200
19201
20287
20435
22637
26839
26840
27898
27899
27920
27921
28075
28076
28105
28106
28107
28108
29514
29530
30916
30955
31184
31218
31345
31512
32400
33131
34321


In [4]:
len(data_final)

15281

In [4]:
data_raw = []
for item in data_final:
    temp = {}
    temp['question'] = item['question']
    temp['query'] = item['query']
    data_raw.append(temp)
data_prep = {}
for item in data_final:
    data_prep[item['query']] = item['logic']
with open('preprocessed/question_sql.json','w') as f:
    json.dump(data_raw,f)
with open('preprocessed/logic.json','w') as f:
    json.dump(data_prep,f)
with open ('preprocessed/alldata.json','w') as f:
    json.dump(data_final,f)